In [57]:
import pandas as pd
import os
import json
import sqlalchemy
import logging
import re

In [94]:
def describe_variable(variable:str):
    null_values = sum(osm_lines[variable].isnull())
    valid_values = num_rows - null_values
    print(f"""NULL values: {null_values}
    Valid values: {valid_values}""")
    print(osm_lines[variable].unique())
    print(osm_lines[variable].value_counts(dropna=False))

In [59]:
# Path variables
logger = logging.getLogger('Crossmap - INSEE to geojson')

target_folder = "/Users/duccioa/CLOUD/01_Cloud/01_Work/04_Projects/0031_CrossMap/05_Data/OSM/"

credential_file_path = '/Users/duccioa/CLOUD/01_Cloud/01_Work/04_Projects/0031_CrossMap/04_Admin/03_Credentials/crossmapDB_credentials.json'

In [60]:
# SQL connection
with open(credential_file_path) as data_file:
    credential_json = json.load(data_file)

server = 'localhost'
credentials = credential_json['crossmap_database_credentials'][server]
host = credentials['host']
port = credentials['port']
database = credentials['database']
user = credentials['user']
password = credentials['password']
driver = credentials['driver']
sql_connection = driver + "://" + user + ":" + password + "@" + host + ":" + port + "/" + database
print(f"Creating connection on {host} with database {database} for user {user}")
engine = sqlalchemy.create_engine(sql_connection)


Creating connection on localhost with database crossmap for user duccioa


# osm_plane_line exploration

Exploration and cleaning of the variables in the table osm_planet_line

In [61]:
osm_lines = pd.read_sql("SELECT * FROM paris.osm_line_idf_view", engine)
print("Done.")

Done.


In [62]:
num_rows = osm_lines.shape[0]
num_columns = osm_lines.shape[1]
print(osm_lines.shape)

(529058, 71)


In [63]:
print(osm_lines.columns)

Index(['osm_id', 'access', 'addr:housename', 'addr:housenumber',
       'addr:interpolation', 'admin_level', 'aerialway', 'aeroway', 'amenity',
       'area', 'barrier', 'bicycle', 'brand', 'bridge', 'boundary', 'building',
       'construction', 'covered', 'culvert', 'cutting', 'denomination',
       'disused', 'embankment', 'foot', 'generator:source', 'harbour',
       'highway', 'historic', 'horse', 'intermittent', 'junction', 'landuse',
       'layer', 'leisure', 'lock', 'man_made', 'military', 'motorcar', 'name',
       'natural', 'office', 'oneway', 'operator', 'place', 'population',
       'power', 'power_source', 'public_transport', 'railway', 'ref',
       'religion', 'route', 'service', 'shop', 'sport', 'surface', 'toll',
       'tourism', 'tower:type', 'tracktype', 'tunnel', 'water', 'waterway',
       'wetland', 'width', 'wood', 'z_order', 'way_area', 'tags', 'way',
       'geom_2154'],
      dtype='object')


In [64]:
print(osm_lines.dtypes)

osm_id                 int64
access                object
addr:housename        object
addr:housenumber      object
addr:interpolation    object
admin_level           object
aerialway             object
aeroway               object
amenity               object
area                  object
barrier               object
bicycle               object
brand                 object
bridge                object
boundary              object
building              object
construction          object
covered               object
culvert               object
cutting               object
denomination          object
disused               object
embankment            object
foot                  object
generator:source      object
harbour               object
highway               object
historic              object
horse                 object
intermittent          object
                       ...  
oneway                object
operator              object
place                 object
population    

In [65]:
osm_lines.head(100)

,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,water,waterway,wetland,width,wood,z_order,way_area,tags,way,geom_2154
0,14032535,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,339,None,"{'lit': 'yes', 'lanes': '2', 'alt_name': 'BDL'...",0102000020110F00000700000085EB51B8F75310416666...,01020000206A08000007000000D040F3ED16072441B8C0...
1,14032865,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,29,None,"{'lit': 'yes', 'lanes': '1', 'destination': 'S...",0102000020110F00000600000048E17A1429601041713D...,01020000206A08000006000000B3D179661A0B24411CC2...
2,14032879,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,329,None,"{'lit': 'yes', 'lanes': '1'}",0102000020110F00000A0000009A99999954561041EC51...,01020000206A0800000A00000059AD17A2DC072441B20A...
3,14032890,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,29,None,"{'lit': 'yes', 'lanes': '2'}",0102000020110F000008000000713D0AD74C5110413D0A...,01020000206A080000080000005D0F5B06330624416943...
4,14033804,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,339,None,"{'lit': 'yes', 'lanes': '2', 'alt_name': 'BDL'...",0102000020110F00000C000000B81E85EBFF5610410AD7...,01020000206A0800000C000000D631677D24082441B9FC...
5,14033808,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,39,None,"{'lit': 'yes', 'lanes': '2', 'alt_name': 'BDL'...",0102000020110F0000030000000AD7A370AA531041295C...,01020000206A080000030000005217603C0807244121CE...
6,14033809,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,36,None,{},0102000020110F0000070000001F85EB51DB4E10410AD7...,01020000206A08000007000000249BD3E770052441F68A...
7,14033811,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,336,None,{'cycleway:right': 'lane'},0102000020110F000004000000CDCCCCCCE95310411F85...,01020000206A080000040000004AC06402200724413755...
8,14033812,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,36,None,{},0102000020110F000006000000B81E85EBF5521041B81E...,01020000206A0800000600000048DD9B5ACE06244123E6...
9,14033814,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,336,None,{'cycleway:right': 'lane'},0102000020110F000003000000F6285C8F5C53104148E1...,01020000206A08000003000000BEB8753EF0062441C9FB...


In [66]:
describe_variable("highway")

NULL values: 105204
    Valid values: 423854
['motorway' 'motorway_link' 'secondary' 'trunk' 'tertiary' 'pedestrian'
 'unclassified' 'primary' 'service' 'residential' None 'trunk_link'
 'primary_link' 'footway' 'track' 'secondary_link' 'tertiary_link'
 'living_street' 'path' 'steps' 'bus_guideway' 'construction' 'cycleway'
 'road' 'bridleway' 'raceway' 'crossing' 'bus_stop'
 'emergency_access_point' 'platform' 'disused' 'access_ramp' 'elevator'
 'proposed' 'virtual' 'yes' 'abandoned' 'corridor' 'services' 'step'
 'traffic_island']
residential               111205
service                    76229
footway                    58991
track                      36101
path                       26789
unclassified               25530
tertiary                   21915
secondary                  20248
primary                    12472
steps                      10916
cycleway                    4851
pedestrian                  4231
living_street               2158
motorway                    2081
m

In [67]:
describe_variable("railway")

NULL values: 510557
    Valid values: 18501
[None 'rail' 'disused' 'tram' 'light_rail' 'subway' 'abandoned'
 'construction' 'funicular' 'platform' 'preserved' 'miniature' 'dismantled'
 'narrow_gauge' 'monorail' 'no' 'turntable' 'traverser' 'razed' 'proposed'
 'train_station_entrance' 'buffer_stop']
rail                      15064
disused                    1176
subway                      531
platform                    486
tram                        483
abandoned                   419
light_rail                  164
narrow_gauge                 53
preserved                    32
construction                 26
razed                        26
monorail                      9
proposed                      8
turntable                     6
dismantled                    6
funicular                     4
traverser                     3
miniature                     2
no                            1
buffer_stop                   1
train_station_entrance        1
Name: railway, dtype: int64


In [68]:
describe_variable("public_transport")

NULL values: 528791
    Valid values: 267
[None 'platform' 'stop_position']
platform         259
stop_position      8
Name: public_transport, dtype: int64


In [69]:
describe_variable("tunnel")

NULL values: 520158
    Valid values: 8900
[None 'yes' 'no' 'building_passage' 'culvert' 'covered']
yes                 7102
culvert              884
building_passage     815
no                    85
covered               14
Name: tunnel, dtype: int64


In [70]:
describe_variable("water")

NULL values: 529057
    Valid values: 1
[None 'pond']
pond    1
Name: water, dtype: int64


In [71]:
describe_variable("waterway")

NULL values: 522448
    Valid values: 6610
[None 'river' 'drain' 'stream' 'ditch' 'canal' 'lock_gate' 'dam'
 'fish_pass' 'weir' 'quay' 'waterfall' 'wadi' 'hitch']
stream       3572
river        1504
drain         609
ditch         473
canal         363
weir           59
dam            12
waterfall       7
lock_gate       6
fish_pass       2
wadi            1
hitch           1
quay            1
Name: waterway, dtype: int64


In [72]:
describe_variable("width")








NULL values: 527177
    Valid values: 1881
[None '15' '5' '2.6 m' '2.3 m' '12' '4' '2' '1.5' '20' '3' '3.0' '2.5' '7'
 '8' '6' '1.7' '3.5' '10' '40' '1' '0.5' '-2' '2.20' '2 m' '1.2' 'narrow'
 '0.8' '0.2' '-1' '0.35' '0.3' '1.8' '0.4' '2.50' '1.3' '30' '0.6' '9'
 '1.50' '0.7' '1.20' '16' '3m' '14' '2.5 m' '1.5 m' '0.30' '43' '2.1' '0.9'
 '5.5' '1.4' '3.4' '70 cm' '2.2' '2.8' '0' '-4' '0.70' '13' '1m' '1 m'
 '0.50' '4.5' '0,9' '80' '45' '60' '136' '100' '44' '50' '7.40' '2.3' '4.7'
 '4.8']
1         489
2         364
3         217
4         124
5          87
1.5        80
0.5        79
0.2        58
2.5        48
6          40
0.3        33
3.5        16
30         15
10         15
1.2        14
45         14
8          14
0.4        13
7          12
12         12
60         11
1.50        9
20          8
0.8         7
15          6
100         5
40          5
1.8         4
1.20        4
0           3
         ... 
1.4         2
4.5         2
1m          1
1 m         1
0.50        1
1.

In [82]:
width = osm_lines['width'].str.findall("([0-9,.]+)")
print(type(width))
len(width)
osm_lines['width'] = width.str.replace(",", ".")
describe_variable('width')

<class 'pandas.core.series.Series'>
NULL values: 529058
    Valid values: 0
[ nan]
Series([], Name: width, dtype: int64)


In [51]:
describe_variable("bicycle")

NULL values: 781319
    Valid values: 36355
[None 'designated' 'yes' 'no' 'dismount' 'permissive' 'discouraged'
 'private' 'destination' 'unsuitable' 'unknown' 'unofficial' 'allowed'
 'opposite' 'track' 'use_sidepath' 'dismounted' 'opposite_lane' 'official'
 'customers' 'limited']
yes              22343
no                6451
designated        2889
limited           1552
dismount          1366
permissive        1362
private            253
unknown             33
destination         31
discouraged         18
allowed             17
unsuitable          13
unofficial          11
use_sidepath         6
official             4
dismounted           2
opposite_lane        1
opposite             1
customers            1
track                1
Name: bicycle, dtype: int64


In [52]:
describe_variable("bridge")

NULL values: 798214
    Valid values: 19460
[None 'yes' 'aqueduct' 'viaduct' 'movable' 'boardwalk' 'no' '1' 'gangway'
 'covered' 'path']
yes          18320
viaduct       1040
no              38
aqueduct        29
movable         16
covered          5
1                4
boardwalk        4
gangway          3
path             1
Name: bridge, dtype: int64


# osm_planet_polygon

In [84]:
osm_poly = pd.read_sql("SELECT * FROM osm.planet_osm_polygon", engine)
print("Done.")

Done.


In [85]:
num_rows = osm_poly.shape[0]
num_columns = osm_poly.shape[1]
print(osm_poly.shape)

(3759000, 71)


In [87]:
print(osm_poly.dtypes)

osm_id                  int64
access                 object
addr:housename         object
addr:housenumber       object
addr:interpolation     object
admin_level            object
aerialway              object
aeroway                object
amenity                object
area                   object
barrier                object
bicycle                object
brand                  object
bridge                 object
boundary               object
building               object
construction           object
covered                object
culvert                object
cutting                object
denomination           object
disused                object
embankment             object
foot                   object
generator:source       object
harbour                object
highway                object
historic               object
horse                  object
intermittent           object
                       ...   
oneway                 object
operator               object
place     

In [88]:
osm_poly.head(100)

,osm_id,access,addr:housename,addr:housenumber,addr:interpolation,admin_level,aerialway,aeroway,amenity,area,...,water,waterway,wetland,width,wood,z_order,way_area,tags,way,geom_centroids
0,132249802,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,50.4555,{},0103000020110F00000100000005000000713D0AD7D14D...,None
1,132249798,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,83.1571,{},0103000020110F00000100000005000000B81E85EB5F4F...,None
2,132249796,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,51.6624,{},0103000020110F00000100000005000000B81E85EB955A...,None
3,132249794,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,93.2963,{'wall': 'no'},0103000020110F000001000000050000000AD7A3702B1A...,None
4,132249790,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,46.2838,{},0103000020110F00000100000005000000000000004233...,None
5,132249783,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,73.8717,{},0103000020110F0000010000000500000033333333CD5D...,None
6,132249781,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,11.3283,{'wall': 'no'},0103000020110F00000100000005000000EC51B81EB775...,None
7,132249777,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,575.6750,{},0103000020110F00000100000005000000AE47E17AFE7D...,None
8,132249771,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,6.2354,{'wall': 'no'},0103000020110F00000100000005000000E17A14AE356C...,None
9,132249769,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,0,44.9800,{},0103000020110F00000100000005000000D7A3703D325B...,None


In [86]:
print(osm_poly.columns)

Index(['osm_id', 'access', 'addr:housename', 'addr:housenumber',
       'addr:interpolation', 'admin_level', 'aerialway', 'aeroway', 'amenity',
       'area', 'barrier', 'bicycle', 'brand', 'bridge', 'boundary', 'building',
       'construction', 'covered', 'culvert', 'cutting', 'denomination',
       'disused', 'embankment', 'foot', 'generator:source', 'harbour',
       'highway', 'historic', 'horse', 'intermittent', 'junction', 'landuse',
       'layer', 'leisure', 'lock', 'man_made', 'military', 'motorcar', 'name',
       'natural', 'office', 'oneway', 'operator', 'place', 'population',
       'power', 'power_source', 'public_transport', 'railway', 'ref',
       'religion', 'route', 'service', 'shop', 'sport', 'surface', 'toll',
       'tourism', 'tower:type', 'tracktype', 'tunnel', 'water', 'waterway',
       'wetland', 'width', 'wood', 'z_order', 'way_area', 'tags', 'way',
       'geom_centroids'],
      dtype='object')


In [89]:
describe_variable("amenity")

NULL values: 528921
    Valid values: 3230079
[None 'bench' 'service' 'bicycle_parking' 'marketplace' 'parking_entrance'
 'place_of_worship' 'hospital' 'recycling' 'parking' 'pier' 'wall'
 'shelter' 'swimming_pool' 'school' 'retirement_home' 'fountain']
bench               92
parking             13
bicycle_parking      8
parking_entrance     6
marketplace          4
shelter              2
service              2
pier                 2
place_of_worship     1
wall                 1
recycling            1
fountain             1
retirement_home      1
hospital             1
swimming_pool        1
school               1
Name: amenity, dtype: int64


In [96]:
describe_variable("building")

NULL values: 528996
    Valid values: 3230004
[None 'roof' 'yes' 'hospital' 'residential' 'no' 'church' 'tower'
 'Sente du Bois de la Ferme' 'house']
NaN                          528996
house                            34
yes                              18
residential                       2
tower                             2
hospital                          2
church                            1
no                                1
roof                              1
Sente du Bois de la Ferme         1
Name: building, dtype: int64


In [95]:
describe_variable("construction")
# print(osm_poly["construction"][:100])

NULL values: 528690
    Valid values: 3230310
[None 'trunk_link' 'yes' 'secondary' 'subway' 'residential' 'unclassified'
 'motorway_link' 'primary' 'minor' 'motorway' 'trunk' 'service' 'path'
 'steps' 'pedestrian' 'footway' 'cycleway' 'tertiary_link' 'primary_link'
 'tertiary' 'true' 'light_rail' 'track' 'living_street' 'footpath' 'road'
 'bridge' 'rail' 'tram' 'pont']
NaN              528690
residential          74
yes                  72
service              27
unclassified         24
secondary            22
primary              19
motorway_link        15
trunk                14
trunk_link           12
pedestrian           11
tertiary             10
footway               9
subway                7
cycleway              7
motorway              6
minor                 6
light_rail            5
steps                 5
tram                  4
rail                  3
living_street         2
primary_link          2
tertiary_link         2
bridge                2
track                 2
path

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
      ... 
70    None
71    None
72    None
73    None
74    None
75    None
76    None
77    None
78    None
79    None
80    None
81    None
82    None
83    None
84    None
85    None
86    None
87    None
88    None
89    None
90    None
91    None
92    None
93    None
94    None
95    None
96    None
97    None
98    None
99    None
Name: construction, Length: 100, dtype: object


In [97]:
describe_variable("landuse")

NULL values: 529011
    Valid values: 3229989
[None 'residential' 'industrial' 'farmland' 'basin' 'construction' 'forest'
 'grass' 'meadow']
NaN             529011
residential         34
forest               4
farmland             3
meadow               2
basin                1
grass                1
construction         1
industrial           1
Name: landuse, dtype: int64
